# ROI Polygon Visualization
Visualize polygon of interest based on stick positions and displacements

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import xarray as xr
%matplotlib inline

In [ ]:
# Load your dataset
ds2 = xr.open_dataset('your_dataset.nc')  # Replace with actual path
# Or use your existing loaded ds2

In [ ]:
# Parameters
trial_idx = 0  # Select which trial to visualize
start_frame = 100  # Starting frame
n_frames = 50  # Number of frames to plot
keypoint_names = ['sticktip', 'stickstripeprox']  # Keypoints for polygon corners

In [ ]:
# Extract positions and displacements for the selected trial and time range
positions = ds2['position'].isel(
    trials=trial_idx,
    time=slice(start_frame, start_frame + n_frames)
)

displacements = ds2['displacement'].isel(
    trials=trial_idx,
    time=slice(start_frame, start_frame + n_frames)
)

print(f"Position shape: {positions.shape}")
print(f"Displacement shape: {displacements.shape}")

In [ ]:
# Create figure with small subplots in a large grid
n_cols = 10
n_rows = int(np.ceil(n_frames / n_cols))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 12))
axes = axes.flatten()

# Hide extra subplots
for ax in axes[n_frames:]:
    ax.axis('off')

# Calculate global axis limits for consistency
all_x = []
all_y = []

for kp in keypoint_names:
    pos_x = positions.sel(keypoints=kp, space='x').values
    pos_y = positions.sel(keypoints=kp, space='y').values
    disp_x = displacements.sel(keypoints=kp, space='x').values
    disp_y = displacements.sel(keypoints=kp, space='y').values
    
    all_x.extend(pos_x.flatten())
    all_x.extend((pos_x + disp_x).flatten())
    all_y.extend(pos_y.flatten())
    all_y.extend((pos_y + disp_y).flatten())

# Remove NaNs and calculate limits
all_x = [x for x in all_x if not np.isnan(x)]
all_y = [y for y in all_y if not np.isnan(y)]

if all_x and all_y:
    x_min, x_max = min(all_x), max(all_x)
    y_min, y_max = min(all_y), max(all_y)
    
    # Add padding
    x_padding = (x_max - x_min) * 0.15
    y_padding = (y_max - y_min) * 0.15
    
    xlim = (x_min - x_padding, x_max + x_padding)
    ylim = (y_min - y_padding, y_max + y_padding)
else:
    xlim = (0, 100)
    ylim = (0, 100)

print(f"X limits: {xlim}")
print(f"Y limits: {ylim}")

In [ ]:
# Plot each frame
for frame_idx in range(n_frames):
    ax = axes[frame_idx]
    
    # Get positions and displacements for this frame
    corners = []
    original_positions = []
    
    for kp_name in keypoint_names:
        # Get position
        pos_x = positions.isel(time=frame_idx).sel(keypoints=kp_name, space='x').values
        pos_y = positions.isel(time=frame_idx).sel(keypoints=kp_name, space='y').values
        
        # Get displacement
        disp_x = displacements.isel(time=frame_idx).sel(keypoints=kp_name, space='x').values
        disp_y = displacements.isel(time=frame_idx).sel(keypoints=kp_name, space='y').values
        
        # Handle multiple individuals by taking mean
        if hasattr(pos_x, 'shape') and len(pos_x.shape) > 0:
            pos_x = np.nanmean(pos_x)
            pos_y = np.nanmean(pos_y)
            disp_x = np.nanmean(disp_x)
            disp_y = np.nanmean(disp_y)
        
        # Store original position
        original_positions.append([pos_x, pos_y])
        
        # Calculate corner (position + displacement)
        corner_x = pos_x + disp_x
        corner_y = pos_y + disp_y
        corners.append([corner_x, corner_y])
    
    corners = np.array(corners)
    original_positions = np.array(original_positions)
    
    # Create 4 corners from 2 diagonal points
    if len(corners) == 2 and not np.any(np.isnan(corners)):
        # Create rectangle corners
        polygon_corners = np.array([
            corners[0],  # Bottom left (sticktip + displacement)
            [corners[1][0], corners[0][1]],  # Bottom right
            corners[1],  # Top right (stickstripeprox + displacement)
            [corners[0][0], corners[1][1]]   # Top left
        ])
        
        # Plot the polygon
        polygon = patches.Polygon(
            polygon_corners,
            closed=True,
            edgecolor='red',
            facecolor='yellow',
            alpha=0.4,
            linewidth=1.5
        )
        ax.add_patch(polygon)
        
        # Plot corner points (position + displacement)
        ax.scatter(corners[:, 0], corners[:, 1],
                  c='blue', s=25, zorder=5, marker='o',
                  label='Corner (pos+disp)' if frame_idx == 0 else '')
        
        # Plot original positions
        ax.scatter(original_positions[:, 0], original_positions[:, 1],
                  c='green', s=20, marker='^', zorder=4,
                  label='Original pos' if frame_idx == 0 else '')
        
        # Draw arrows from original to displaced positions
        for i in range(len(original_positions)):
            if not np.any(np.isnan([original_positions[i], corners[i]])):
                ax.arrow(original_positions[i, 0], original_positions[i, 1],
                        corners[i, 0] - original_positions[i, 0],
                        corners[i, 1] - original_positions[i, 1],
                        head_width=2, head_length=1,
                        fc='gray', ec='gray', alpha=0.5, linewidth=0.5)
    
    # Set axis properties
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_aspect('equal')
    ax.set_title(f'Frame {start_frame + frame_idx}', fontsize=7)
    ax.tick_params(labelsize=5)
    ax.grid(True, alpha=0.2, linewidth=0.5)
    
    # Add legend to first subplot
    if frame_idx == 0:
        ax.legend(fontsize=5, loc='upper right')

plt.suptitle(f'ROI Polygon Animation - Trial {trial_idx} (Frames {start_frame}-{start_frame+n_frames})',
            fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## Single Frame Detail View

In [ ]:
# Plot a single frame in detail
selected_frame = start_frame + 25  # Middle frame

fig, ax = plt.subplots(figsize=(8, 8))

# Get data for selected frame
frame_positions = ds2['position'].isel(trials=trial_idx, time=selected_frame) # fix old isel trial format?
frame_displacements = ds2['displacement'].isel(trials=trial_idx, time=selected_frame)

corners = []
original_positions = []
labels = []

for kp_name in keypoint_names:
    # Get position and displacement
    pos_x = frame_positions.sel(keypoints=kp_name, space='x').values
    pos_y = frame_positions.sel(keypoints=kp_name, space='y').values
    disp_x = frame_displacements.sel(keypoints=kp_name, space='x').values
    disp_y = frame_displacements.sel(keypoints=kp_name, space='y').values
    
    # Handle multiple individuals
    if hasattr(pos_x, 'shape') and len(pos_x.shape) > 0:
        pos_x = np.nanmean(pos_x)
        pos_y = np.nanmean(pos_y)
        disp_x = np.nanmean(disp_x)
        disp_y = np.nanmean(disp_y)
    
    original_positions.append([pos_x, pos_y])
    corners.append([pos_x + disp_x, pos_y + disp_y])
    labels.append(kp_name)

corners = np.array(corners)
original_positions = np.array(original_positions)

if len(corners) == 2 and not np.any(np.isnan(corners)):
    # Create polygon
    polygon_corners = np.array([
        corners[0],
        [corners[1][0], corners[0][1]],
        corners[1],
        [corners[0][0], corners[1][1]]
    ])
    
    polygon = patches.Polygon(
        polygon_corners,
        closed=True,
        edgecolor='red',
        facecolor='yellow',
        alpha=0.3,
        linewidth=2,
        label='Nest region (ROI)'
    )
    ax.add_patch(polygon)
    
    # Plot points and labels
    for i, (orig, corner, label) in enumerate(zip(original_positions, corners, labels)):
        # Original position
        ax.scatter(orig[0], orig[1], c='green', s=100, marker='^', zorder=5)
        ax.annotate(f'{label}\n(original)', xy=(orig[0], orig[1]),
                   xytext=(5, 5), textcoords='offset points', fontsize=8)
        
        # Displaced position
        ax.scatter(corner[0], corner[1], c='blue', s=100, marker='o', zorder=5)
        ax.annotate(f'{label}\n(+displacement)', xy=(corner[0], corner[1]),
                   xytext=(5, -15), textcoords='offset points', fontsize=8)
        
        # Arrow
        ax.arrow(orig[0], orig[1],
                corner[0] - orig[0], corner[1] - orig[1],
                head_width=3, head_length=2,
                fc='gray', ec='gray', alpha=0.6, linewidth=1.5)

ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.set_xlabel('X position', fontsize=10)
ax.set_ylabel('Y position', fontsize=10)
ax.set_title(f'ROI Polygon Detail - Trial {trial_idx}, Frame {selected_frame}', fontsize=12)
ax.legend(loc='best')

plt.tight_layout()
plt.show()

## Trajectory Over Time

In [ ]:
# Plot polygon evolution over time with color gradient
fig, ax = plt.subplots(figsize=(10, 10))

# Create colormap for time
colors = plt.cm.viridis(np.linspace(0, 1, n_frames))

for frame_idx in range(0, n_frames, 2):  # Plot every 2nd frame for clarity
    corners = []
    
    for kp_name in keypoint_names:
        pos_x = positions.isel(time=frame_idx).sel(keypoints=kp_name, space='x').values
        pos_y = positions.isel(time=frame_idx).sel(keypoints=kp_name, space='y').values
        disp_x = displacements.isel(time=frame_idx).sel(keypoints=kp_name, space='x').values
        disp_y = displacements.isel(time=frame_idx).sel(keypoints=kp_name, space='y').values
        
        if hasattr(pos_x, 'shape') and len(pos_x.shape) > 0:
            pos_x = np.nanmean(pos_x)
            pos_y = np.nanmean(pos_y)
            disp_x = np.nanmean(disp_x)
            disp_y = np.nanmean(disp_y)
        
        corners.append([pos_x + disp_x, pos_y + disp_y])
    
    corners = np.array(corners)
    
    if len(corners) == 2 and not np.any(np.isnan(corners)):
        polygon_corners = np.array([
            corners[0],
            [corners[1][0], corners[0][1]],
            corners[1],
            [corners[0][0], corners[1][1]]
        ])
        
        alpha = 0.1 + 0.6 * (frame_idx / n_frames)
        polygon = patches.Polygon(
            polygon_corners,
            closed=True,
            edgecolor=colors[frame_idx],
            facecolor='none',
            alpha=alpha,
            linewidth=1
        )
        ax.add_patch(polygon)

# Add colorbar
sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis,
                           norm=plt.Normalize(vmin=start_frame,
                                            vmax=start_frame+n_frames))
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, label='Frame number')

ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.set_xlabel('X position', fontsize=10)
ax.set_ylabel('Y position', fontsize=10)
ax.set_title(f'ROI Polygon Trajectory - Trial {trial_idx}\nFrames {start_frame}-{start_frame+n_frames}',
            fontsize=12)

plt.tight_layout()
plt.show()